# Overview

In this assignment we will take two very large steps forward in understanding how to implement a neural network algorithm. In the first part, we will review some *very basic* concepts related to math and matrix algrebra, just enough so that we can translate what we learned in lectures 1 and 2 into a mathematic formulation. In the second part we will introduce the TensorFlow library, maintained by the Google Brain team, to actually design and train our basic classifier.

### Math and Matrix Algebra Review

* matrix multiplication
* softmax scores
* cross-entropy

### TensorFlow

* overview of graphs
* training a model
* inference from a model

## Restarting your Virtual Machine

If at any point during this assignment you accidentally execute code or do something that cannot seem to undo and need to "restart" the system (including deleting all temporary folders), go ahead and run the following single line of code. It will take about 1 minute to restart. Following this, you will have to proceed at the beginning of the assignment to re-downloaded the data and run the code you have written. Note, the code that you have already written will **not** be deleted; you simply need to start executing the code once again from the start.

In [0]:
!kill -9 -1 # Warning this restarts your machine

## Downloading the Data

The following commands can be used to copy over the assignment materials to your local Colaboratory instance and unzip in preparation for your assignment:

In [0]:
!git clone https://github.com/CAIDMRes/lecture_02
!unzip lecture_02/data.zip
!rm -r lecture_02
!ls



# Math and Matrix Algebra

## Foundations

To review the structure of a neural network, let us keep in mind the diagramatic representation of a neural network covered  in lecture 2:

![Diagramtric Representation](https://raw.githubusercontent.com/CAIDMRes/images/master/assignment_02-01.png)

Recall that each reach "retinal neuron" recieving light from the image is connected to one of ten outputs representing the likelihood that the image represents one of ten possible digits. The neuron with the largest number, also known as **logit score**, gets to decide what the final image is most likely to represent.

Also, recall that each connection itself is modeled by a weight value that represents how strong (or weak) the connection is, and that each of the 784 connections to one of our ten output neurons can in fact be represented as a matrix of numbers:

![Matrix Representation](https://raw.githubusercontent.com/CAIDMRes/images/master/assignment_02-02.png)

Of course, for each of our ten digits, we will have a different 28 x 28 weight matrix for a total of 10 weight matrices. Finally keep in mind that for the sake of ease in representation (given our simple model) both our 28 x 28 input matrix and ten 28 x 28 weight matrices can be **flattened** to just a single 784 x 1 (or 1 x 784) matrix. 

## NumPy Matrix Algebra

A 2D matrix is composed of **rows** and **columns**. The standard notation for referencing the size of a matrix is (# of rows) x (# of columns.) Let's start with a hypothetical toy 3 x 3 image:

In [0]:
import numpy as np

im = np.array([
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 9]
])

print(im)

Recall now the basic rules for matrix multiplication:

![Matrix Multiplication](https://www.mathsisfun.com/algebra/images/matrix-multiply-b.svg)

1. Matrix **A** can be multipled by matrix **B** if the number of columns in A = number of rows in B. In the above example a 2 x 3 matrix can be multipled by a 3 x 2 because the second number (columns) in **A** is equal to the first number (rows) in **B**.
2. Matrix **A** multipled by matrix **B** results in an output matrix **C** with a size that is equal to the (number of rows in **A**) x (number of columns in **B**). In the above example, the result of a 2 x 3 matrix multipled by a 3 x 2 matrix is 2 x 2.
3. To arrive at the (i, j)-th entry in matrix **C**, multiply and add together the i-*th* row in **A** by the j-*th* column in **B**. In the above example the entry in the first row / second column of **C** (also referenced as C[1, 2]) can be obtained by multiplying then adding the first row in **A** by the second column in **B**:
```
C[1,2] = 1*8 + 2*10 + 3*12 = 8 + 20 + 36 = 64
```

Given this, what is the output shape and result of im x im?



In [0]:
# Use np.matmul() for matrix multiply
c = np.matmul(im, im)

print(c.shape)

print(c)

## Using MNIST data

Let's go ahead and load up the MNIST dataset from lecture 2. Recall that `x` is a matrix containing all the images and `y` is a matrix containing all the labels.

In [0]:
# Loading a pickle (*.pkl) file
import pickle
x = pickle.load(open('x.pkl', 'rb'))

# x is a NumPy array with (flattened) image data
print(type(x))
print(x.shape)

# y is a NumPy array with labels
y = pickle.load(open('y.pkl', 'rb'))
print(y.shape) 

As we see here `x` is a (60,000 x 784) matrix with all our imaging data. As we learned in the previous section, this matrix contains 60,000 rows and 784 columns; each row represents one image (e.g. `x[0], x[1], x[2], ...` etc). Let's take a look at the first image / label in the dataset:

In [0]:
# This is our first image
im = x[0]
im = im.reshape(28, 28)

# This is our first label
label = y[0]

# Let's visualize
import pylab
pylab.imshow(im)
pylab.axis('off')
pylab.title('This is an example of the digit ' + str(label))
pylab.show()


## Matrix multiply

As described in above, each one of these input "retinal neurons" (one for each pixel in the image) gets connected to one of ten output neurons via a weight. Let's consider for example the single output neuron in charge of predicting that the number is zero. This neuron synthesizes the input of a total of 784 weights, one for each pixel, and combines them all into one final **logit score**. This can be implemented via a simple matrix multiply. Let's define the following:

In [0]:
# This is our first image
im = x[0]

# This is 784 random weights from a distribution with mean = 0, SD = 0.0001 
weights = np.random.normal(loc=0, scale=0.0001, size=(784))

# Let's look at the first 10 values of each
print(im[:10])
print(weights[:10])

As expected, the first few values of an image matrix generally contain zeros (e.g. edge of the image), while the first few weights that we initialized randomly contain numbers drawn from a normal Gaussian distribution (bell curve) with a mean of 0 and standard deviation of 0.0001.

How do we multiply these two matrices together and sum up the values so that we end up with just *one* final logit score? Recall that both matrices are (784 x 1) in size, so that breaks the rule for having matrix sizes properly match for multiplication (recall **columns** of A must = **rows** of B)? Do we need to do a (1 x 784) x (784 x 1) operation or (784 x 1) x (1 x 784) operation? 

The answer is that you need to a  (1 x 784) x (784 x 1). This as you remember, based on the rules of matrix multiplication, will result in each pixel value being multiplied by it's corresponding weight and summing the results up into a single cumulative value, which is exactly what we want. To code this in NumPy:

In [0]:
A = im.reshape(1, 784)
B = weights.reshape(784, 1)
logit = np.matmul(A, B)
print(logit)

What if we want to define ten total weight matrices? Instead of doing this ten different times, you can simply stack ten **rows** of matrices on top of each other, yielding a combined weight matrix of size 784 x 10. Then when we multiply our original (1 x 784) image by this combined weight matrix, we get *ten* different values representing the likelihood of each corresponding digit. If you need, refer back to the diagram for matrix multiplication above to prove to yourself this is what is happenening. To code this in NumPy:

In [0]:
# This is 784 random weights from a distribution with mean = 0, SD = 0.0001 
weights = np.random.normal(loc=0, scale=0.0001, size=(784, 10))

# One matrix multiply to get the logit score for all weights at one time
logits = np.matmul(A, weights)

# Predictions is (1 x 10) in size; lets see what's inside
for i in range(10):
    print('The logit score for being a digit %i is: %.03f' % (i, logits[0, i]))

## Softmax

To translate raw score values into probabilities (likelihood between 0 and 1) we use a special formula known as the softmax. The softmax score is calculated as follows:

1. For each logits score ***x***, convert that value to ***e ^ x*** (where ***e*** is the natural log base). This will represent that score **numerator**.
2. Add up all the converted scores. This will represent the score **denominator**.
3. Calculate each score by taking the numerator / denominator. If done correctly, all scores should add up to 1 (e.g. a valid probability distribution).

Let's write the a softmax method together:

In [0]:
def softmax(logits):
    """
    Method to convert raw logit scores into softmax probabilities
    
    :params
    
      (np.array) logits: a 1 x N array representing N total predictions
    
    """
    # (1) Convert the predictions array to exponential version
    # HINT: use np.exp()
    e = ?
    
    # (2) Find the total sum of all exponentials
    s = ?
    
    # (3) Take the exponential version and divide it by the total sum
    prob = ?
    
    # (4) Return your result
    return prob
    
s = softmax(logits)
for i in range(10):
    print('The prob for being a digit %i is: %.03f' % (i, s[0, i]))
    
print('The sum of all probabilities is: %.03f' % np.sum(s))

Given that the output `s`, how do I find out the index for the highest probability? (HINT: use `np.argmax()`)

In [0]:
# Find index within s containing the highest prob
?

# TensorFlow

## Understanding graphs

Now we're set to convert this knowledge into TensorFlow code to train a basic classifier! To start, we need to understand just a little bit about how TensorFlow works. To train network, we first need to define the necessary operations in a structure known as a **graph**. A graph simply represents the series of instructions that dictate what happens to our input data as it *flows* through the graph. Keep in mind that while defining a graph, nothing is actually being calculated yet; we are simply creating a template or blueprint for what *should* happen once data starts to be passed through the graph.

## Placeholders

To begin, we first define what is known as a **placeholder**. A **placeholder** defines where data will be placed into the graph. To define a **placeholder** we must know:

1. The type of data we're using. The two most popular types of data are *floating point* (e.g. decimals) or *integers*. Since neural network weights are finely tuned, they are usually represented by *floating point* decimals, and will be the type of data we use for all our projects.
2. The size of data we're using. Sometimes the size along a particular matrix dimension may be unknown, in which case we can use the Python `None` variable.

Let's see a few examples:

In [0]:
import tensorflow as tf

# Integer matrix of size 4 x 2
im = tf.placeholder(tf.int32, [4, 2])

# Floating point matrices of size 3 x 3
im = tf.placeholder(tf.float32, [3, 3])

# Floating point matrices of ? x 3 size
im = tf.placeholder(tf.float32, [None, 3])

# Floating point matrices of size 1 x 10
im = tf.placeholder(tf.float32, [1, 10])

## Variables

Now let's define a single variable for weights. This variable is defined using `tf.Variable()` instead of a placeholder because we want TensorFlow to keep track of this variable for us and manipulate it over time (e.g. change the weights values as the network is learning). By contrast, the placeholder is something we need to provide to the model and which stays constant during the learning process. 

To define a new variable, simply pass the value of whatever out want the new variable to be initialized as:

In [0]:
# Define weights in Numpy
weights = np.random.normal(loc=0, scale=0.0001, size=(10, 5))

# Define weights in Tensorflow
weights = tf.Variable(weights.astype('float32'))

print(weights.shape)

## Matrix multiply

The syntax for a matrix multiply is very similar in TensorFlow compared by NumPy:

In [0]:
logits = tf.matmul(im, weights)

What is size of the resulting logits matrix?

In [0]:
print(logits.shape)

## Running a graph

Now that we've defined a simple "graph" (composing of a grand total of one operation), let's go ahead and "run" this model. To do so, we need to tell TensorFlow to create the graph for us. This is done by creating a `tf.InteractiveSession()` object and initializing all the variables:

In [0]:
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()

Alright, now we have a graph ready for us to interact with. To pass data through the graph (e.g. let our input matrices/tensors *flow* through the graph) we use the `sess.run()` method tells us which output in the graph we would like to extract. This method requires the creation of what is known as a **feed_dict** (a Python dictionary) where each dictionary *key* contains a placeholder variable, and the corresponding dictionary *value* contains the data you would like to place inside that placeholder. In our simple example, the only placeholder we have is the `im` variable which we defined as a *float* matrix of size 1 x 10 (see above). An example **feed_dict** dictionary would thus look like this:

In [0]:
# Let's feed a random number image
feed_dict = {im: np.random.rand(1, 10)}

To use `sess.run()`, simply include:

1. The output(s) in the graph you would like to extract.
2. A valid **feed_dict**.

In [0]:
# Extract the input image
output = sess.run(im, feed_dict)
print('This is the input image')
print(output)

# Extract the weights
output = sess.run(weights, feed_dict)
print('This is the weights')
print(output)

# Extract the logits 
print('This is the logits')
output = sess.run(logits, feed_dict)
print(output)

# Extract all values in one call
outputs = sess.run([im, weights, logits], feed_dict)
print('This is the input image')
print(outputs[0])
print('This is the input weights')
print(outputs[1])
print('This is the logits')
print(outputs[2])

## Defining Loss

The last thing we need to do is to mathematically define how good our model weights are at any given time. We will do this with a formula known as the softmax cross-entropy function, which is really much simpler than it sounds. Recall that softmax probablities from above are simply the conversion of raw logit scores into a valid probability distribution (sum equals 1). For example, a valid softmax probability distribution may be:

```
softmax = [0.2, 0.3, 0.5]
```

If out of three possible outcomes the last one is the correct answer, then the *optimal* target distribution is:

```
target = [0.0, 0.0, 1.0]
```

To quantify how good these two distributions match, we use `tf.losses.sparse_softmax_cross_entropy(labels, logits)` where labels represent the correct answer (0, 1, 2, ... etc) and logits represent our raw logits scores. The lower this number, the better our prediction is!

To see all of this, let's try building another graph:

In [0]:
# Reset our graph to build a new one
tf.reset_default_graph()

# Define placeholders for both input image and label
im = tf.placeholder(tf.float32, [1, 10])    # example 1 x 10 test images
labels = tf.placeholder(tf.int64, [1])      # example single value labels

# Define our weights
weights = np.random.normal(loc=0, scale=1, size=(10, 5))
weights = tf.Variable(weights.astype('float32'))

# Define our matmul operation
logits = tf.matmul(im, weights)

# Define our loss
loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

# Initialize our test graph
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()

# Run with 10 random test examples
for i in range(10):
    
    # Generate random input data and label
    rand_im = np.random.rand(1, 10)              # generate random 1 x 10 image
    rand_labels = np.random.randint(5, size=(1)) # generate random label between 0 and 4
    
    # Convert to types matching our defined placeholders
    rand_im = rand_im.astype('float32')
    rand_labels = rand_labels.astype('int64')
    
    # Prepare feed_dict
    feed_dict = {im: rand_im, labels: rand_labels}
    
    # Calculate logits and loss  
    outputs = sess.run([logits, loss], feed_dict)
    
    # Use argmax to determine highest logit (model guess)
    argmax = np.argmax(outputs[0])
    
    # Print
    print('The answer was: %i | The guess was: %i | The loss was: %0.4f' %
         (rand_labels, argmax, outputs[1]))
    

As you can see, correct guesses are rewarded with a lower loss value, whereas incorrect guesses have a higher loss value.

# Building a Model

Alright, we now have all the tools to build a classifier together! Let's get started.

First, let's create our graph:

In [0]:
def create_model():
    
    # Reset our graph to build a new one
    tf.reset_default_graph()

    # ------------------------------------------------------------------------
    # Define placeholders for our images and labels
    # ------------------------------------------------------------------------
    #
    # 1. Define images
    # 
    #  - type: float32
    #  - size: [None, 784] so that we can feed in as many images as we need
    # 
    # 2. Define labels
    # 
    #  - type: int64
    #  - size: [None] so that we can feed in as many labels as we need
    # 
    # ------------------------------------------------------------------------

    im = tf.placeholder(?)
    labels = tf.placeholder(?)

    # ------------------------------------------------------------------------
    # Define our weights
    # ------------------------------------------------------------------------
    # 
    # 1. Initialize from random distribution with mean = 0, SD = 0.001
    # 2. Convert to float32
    # 3. Move from NumPy to Tensorflow with tf.Variable()
    # 
    # HINT: what size matrix do we need to connect an input image to 10 outputs?
    # 
    # ------------------------------------------------------------------------

    weights = np.random.normal(?)
    weights = tf.Variable(weights.astype(?))

    # ------------------------------------------------------------------------
    # Define our matmul operation
    # ------------------------------------------------------------------------

    logits = tf.matmul(?)

    # ------------------------------------------------------------------------
    # Define our softmax cross-entropy loss
    # ------------------------------------------------------------------------
    # 
    # HINT: use tf.losses.sparse_softmax_cross_entropy() as above
    #
    # ------------------------------------------------------------------------

    loss = tf.losses.sparse_softmax_cross_entropy(?)

    # ------------------------------------------------------------------------
    # Define our optimizer
    # ------------------------------------------------------------------------
    # 
    # An optimizer is a special TensorFlow class that takes your model weights and 
    # adjusts them ever so slightly so that they will make a better prediction the
    # next time around. They are implemented with a technique known as 
    # backpropogation which we will learn about in further detail during later 
    # lectures. For now, just know that this is what we are using here.
    #
    # ------------------------------------------------------------------------

    train_op = tf.train.GradientDescentOptimizer(0.001).minimize(loss)
    
    return im, labels, weights, logits, loss, train_op

# ------------------------------------------------------------------------
# Test our model
# ------------------------------------------------------------------------
# 
# If the graph was defined properly, we should be able to check the out
# what the model outputs should look like. Can you guess by the shapes
# of our logits and losses will be?
# 
# ------------------------------------------------------------------------

im, labels, weights, logits, loss, train_op = create_model()
print(logits.shape)
print(loss.shape)

In [0]:
# ------------------------------------------------------------------------
# Create our model
# ------------------------------------------------------------------------

im, labels, weights, logits, loss, train_op = create_model()

# ------------------------------------------------------------------------
# Initialize our test graph
# ------------------------------------------------------------------------
# 
# What two things do we need to initialize our graph?
# 
# ------------------------------------------------------------------------

sess = tf.InteractiveSession()
tf.global_variables_initializer().run()

# ------------------------------------------------------------------------
# Train our algorithm 
# ------------------------------------------------------------------------
# 
# Let's set up a loop to train our algorithm by feeding it data iteratively.
# For each iteration, we will feed a batch_size number of images into our 
# model and let it readjust it's neuronal weights.
# 
# ------------------------------------------------------------------------

iterations = 2000 
batch_size = 256 
accuracies = []
losses = []

for i in range(iterations):
    
    # --------------------------------------------------------------------
    # Grab a total of batch_size number of random images and labels 
    # --------------------------------------------------------------------
    # 
    # 1. Pick batch_size number of random indices between 0 and 60,000
    # 2. Select those images / labels
    #
    # --------------------------------------------------------------------
    
    rand_indices = np.random.randint(?, size=?)
    x_batch = x[rand_indices]
    y_batch = y[rand_indices]
    
    # --------------------------------------------------------------------
    # Normalize x_batch
    # --------------------------------------------------------------------
    # 
    # Currently, values in x range from 0 to 255. If we normalize these values
    # to a mean of 0 and SD of 1 we will improve the stability of training
    # and furthermore improve interpretation of learned weights. Use the
    # following code to normalize your batch:
    # 
    # --------------------------------------------------------------------
    
    x_batch = (x_batch - np.mean(x_batch)) / np.std(x_batch)
    
    # Convert to types matching our defined placeholders
    # HINT: integer, float... ?
    x_batch = x_batch.astype(?)
    y_batch = y_batch.astype(?)
    
    # Prepare feed_dict
    feed_dict = {?}
     
    # --------------------------------------------------------------------
    # Run training iteration via sess.run()
    # --------------------------------------------------------------------
    # 
    # Here, in addition to whichever ouputs we wish to extract, we need to
    # also include the train_op variable. Including train_op will tell 
    # Tensorflow that in addition to calculating the intermediates of our graph,
    # we also need to readjust the variables so that the overall loss goes
    # down.
    # 
    # --------------------------------------------------------------------
    
    outputs = sess.run([logits, loss, train_op], feed_dict)
    
    # --------------------------------------------------------------------
    # Use argmax to determine highest logit (model guess)
    # --------------------------------------------------------------------
    # 
    # Keep in mind our logits matrix is (batch_size x 10) in size representing
    # a total of batch_size number of predictions. How do we process this matrix
    # with the np.argmax() to find the highest logit along each row of the matrix
    # (e.g. find the prediction for each of our images)?
    # 
    # HINT: what does the axis parameter in np.argmax(a, axis) specify?
    # 
    # --------------------------------------------------------------------
    
    predictions = np.argmax(?)
    
    # --------------------------------------------------------------------
    # Calculate accuracy 
    # --------------------------------------------------------------------
    # 
    # Consider the following:
    # 
    # - predictions = the predicted digits
    # - y_batch = the ground-truth digits
    # 
    # How do I calculate an accuracy % with this data?
    # 
    # --------------------------------------------------------------------
    
    accuracy = ? 
    
    # --------------------------------------------------------------------
    # Accumulate and print iteration, loss and accuracy 
    # --------------------------------------------------------------------
    
    # This line of code will print progress
    print('Iteration %05i | Loss = %07.3f | Accuracy = %0.4f' %
        (i + 1, outputs[1], accuracy))
    
    # Append outputs[1] to losses list
    losses.append(outputs[1])
    
    # Append accuracy to accuracies list
    accuracie.append(accuracy)
    
# --------------------------------------------------------------------
# Graph outputs and accuracy
# --------------------------------------------------------------------
pylab.plot(losses)
pylab.title('Model loss over time')
pylab.show()

pylab.plot(accuracies)
pylab.title('Model accuracy over time')

Congratulations! If that went well, you should have trained a network to predict digits. Even though it was extremely simple, it was able to predict with about 90% accuracy. That's really impressive, considering how difficult this would be with hand-crafted rules.

## Model weights

Recall our hypothesis about the final "appearance" of our trained model weights, which are in essence expected to learn what amounts to an *average filter* of the training data. Does this hold true?

In [0]:
# --------------------------------------------------------------------
# Extract model weights 
# --------------------------------------------------------------------
# 
# What line of code do we need here to extract the model weights?
# 
# HINT: what do we pass to sess.run()?
#
# --------------------------------------------------------------------

W = sess.run(?)

# Use this code to visualize weights
fig = pylab.figure()
for i in range(10):
    fig.add_subplot(2, 5, i + 1)
    pylab.axis('off')
    pylab.imshow(W[..., i].reshape(28, 28))
    
pylab.show()